In [1]:
import pandas as pd

attendance = pd.read_csv('data/attendance.csv')
entity_schedule = pd.read_csv('data/entity_schedule.csv')
link_atrraction_park = pd.read_csv('data/link_attraction_park.csv')
waiting_times = pd.read_csv('data/waiting_times.csv')
weather_data = pd.read_csv('data/weather_data.csv')

In [2]:
waiting_times.ENTITY_DESCRIPTION_SHORT.unique()

array(['Roller Coaster', 'Bumper Cars', 'Rapids Ride', 'Crazy Dance',
       'Skyway', 'Free Fall', 'Monorail', 'Swing Ride', 'Crazy Bus',
       'Drop Tower', 'Spinning Coaster', 'Scooby Doo', 'Superman Ride',
       'Spiral Slide', 'Inverted Coaster', 'Water Ride', 'Power Tower',
       'Top Spin', 'Log Flume', 'Oz Theatre', 'Circus Train',
       'Giant Wheel', 'Kiddie Coaster', 'Bungee Jump', 'Zipline',
       'Aeroplane Ride', 'Haunted House', 'Reverse Bungee', 'Go-Karts',
       'Dizzy Dropper', 'Merry Go Round', 'Flying Coaster', 'Gondola',
       'Pirate Ship', 'Giga Coaster', 'Himalaya Ride', 'Sling Shot',
       'Vertical Drop', 'Tilt-A-Whirl'], dtype=object)

In [3]:
roller_coaster = waiting_times[waiting_times['ENTITY_DESCRIPTION_SHORT'] == 'Roller Coaster']
roller_coaster_day = roller_coaster[roller_coaster['WORK_DATE'] == '2019-06-01']
roller_coaster_day.sort_values(by="DEB_TIME")

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
1069315,2019-06-01,2019-06-01 09:00:00.000,9,2019-06-01 09:15:00.000,Roller Coaster,0,1.0000,0.0,0.0,0.0,0,0,0,2.0
1070084,2019-06-01,2019-06-01 09:15:00.000,9,2019-06-01 09:30:00.000,Roller Coaster,0,1.0000,0.0,0.0,0.0,0,0,0,2.0
1069081,2019-06-01,2019-06-01 09:30:00.000,9,2019-06-01 09:45:00.000,Roller Coaster,0,1.0000,0.0,0.0,0.0,0,0,0,2.0
1070096,2019-06-01,2019-06-01 09:45:00.000,9,2019-06-01 10:00:00.000,Roller Coaster,0,1.0000,0.0,0.0,0.0,0,0,0,2.0
1070073,2019-06-01,2019-06-01 10:00:00.000,10,2019-06-01 10:15:00.000,Roller Coaster,20,1.0000,26.0,75.0,37.5,15,15,0,2.0
1069842,2019-06-01,2019-06-01 10:15:00.000,10,2019-06-01 10:30:00.000,Roller Coaster,20,1.8000,45.0,75.0,67.5,15,15,0,2.0
1069088,2019-06-01,2019-06-01 10:30:00.000,10,2019-06-01 10:45:00.000,Roller Coaster,15,2.0000,37.0,75.0,75.0,15,15,0,2.0
1069118,2019-06-01,2019-06-01 10:45:00.000,10,2019-06-01 11:00:00.000,Roller Coaster,40,1.6667,14.0,75.0,62.5,15,15,0,2.0
1069607,2019-06-01,2019-06-01 11:00:00.000,11,2019-06-01 11:15:00.000,Roller Coaster,40,1.0000,20.0,75.0,37.5,15,15,0,2.0
1068969,2019-06-01,2019-06-01 11:15:00.000,11,2019-06-01 11:30:00.000,Roller Coaster,40,1.0000,54.0,75.0,37.5,15,15,0,2.0


In [4]:
import numpy as np

import numpy as np

waiting_times = pd.read_csv('data/waiting_times.csv')

# Filtering for only Port Aventura rides
filtered_attractions = link_atrraction_park[link_atrraction_park["ATTRACTION;PARK"].str.contains("PortAventura World")]
ride_names = filtered_attractions["ATTRACTION;PARK"].str.split(";", expand=True)[0]
waiting_times = waiting_times[waiting_times["ENTITY_DESCRIPTION_SHORT"].isin(ride_names)]

# Processing weather data
relevant_weather_data = weather_data[['dt_iso', 'temp', 'humidity', 'wind_speed', 'clouds_all', 'rain_1h', 'snow_1h']]
relevant_weather_data['dt_iso'] = relevant_weather_data['dt_iso'].str.split('+').str[0]
relevant_weather_data['date'] = pd.to_datetime(relevant_weather_data['dt_iso'].str.split(' ').str[0])
relevant_weather_data['time'] = relevant_weather_data['dt_iso'].str.split(' ').str[1]
relevant_weather_data['hour'] = relevant_weather_data['time'].str.split(':').str[0].astype(int) + 1
relevant_weather_data = relevant_weather_data.drop(columns=['dt_iso', 'time'])
relevant_weather_data.fillna(0, inplace=True)

# One hot encoding times
waiting_times['DEB_TIME'] = waiting_times['DEB_TIME'].astype(str)
waiting_times['date'] = pd.to_datetime(waiting_times['DEB_TIME'].str.split(' ').str[0])
waiting_times['time'] = waiting_times['DEB_TIME'].str.split(' ').str[1]

# Encode the hour and quarter linearly in the time column
waiting_times['hour'] = waiting_times['time'].str.split(':').str[0].astype(int)
waiting_times['minute'] = waiting_times['time'].str.split(':').str[1].astype(int)
waiting_times['time_encoded'] = waiting_times['hour'] * 4 + waiting_times['minute'] / 15

# Merge the weather data
waiting_times = pd.merge(waiting_times, relevant_weather_data, on=['date', 'hour'], how='left')

C:\Users\charl\AppData\Local\Temp\ipykernel_17488\2384680601.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_weather_data['dt_iso'] = relevant_weather_data['dt_iso'].str.split('+').str[0]
C:\Users\charl\AppData\Local\Temp\ipykernel_17488\2384680601.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_weather_data['date'] = pd.to_datetime(relevant_weather_data['dt_iso'].str.split(' ').str[0])
C:\Users\charl\AppData\Local\Temp\ipykernel_17488\2384680601.py:16: SettingWithCopyWarning: 
A

In [5]:
waiting_times['day_of_week'] = waiting_times['date'].dt.dayofweek
waiting_times['day_of_week_sin'] = np.sin(2 * np.pi * waiting_times['day_of_week'] / 7)
waiting_times['day_of_week_cos'] = np.cos(2 * np.pi * waiting_times['day_of_week'] / 7)

waiting_times['year'] = waiting_times['date'].dt.year
waiting_times['month'] = waiting_times['date'].dt.month
waiting_times['day'] = waiting_times['date'].dt.day
waiting_times = pd.get_dummies(waiting_times, columns=['year', 'month', 'day'])

waiting_times.drop(columns=['DEB_TIME', 'time', 'hour', 'minute', 'day_of_week'], inplace=True)

In [6]:
# One hot encoding ride names
ride_names = waiting_times['ENTITY_DESCRIPTION_SHORT'].unique()
ride_names_encoded = pd.get_dummies(waiting_times['ENTITY_DESCRIPTION_SHORT'])
waiting_times = pd.concat([waiting_times, ride_names_encoded], axis=1)

In [7]:
# Dropping any rows with downtime for training
waiting_times = waiting_times[waiting_times["DOWNTIME"] == 0]

# Dropping columns that are not useful for training
waiting_times.drop(columns=["WORK_DATE", "FIN_TIME", "NB_UNITS", "GUEST_CARRIED", "NB_MAX_UNIT",
                            "ADJUST_CAPACITY", "OPEN_TIME", "UP_TIME", "CAPACITY", "DOWNTIME",
                            "ENTITY_DESCRIPTION_SHORT"], inplace=True)

In [8]:
waiting_times.columns

Index(['DEB_TIME_HOUR', 'WAIT_TIME_MAX', 'date', 'time_encoded', 'temp',
       'humidity', 'wind_speed', 'clouds_all', 'rain_1h', 'snow_1h',
       'day_of_week_sin', 'day_of_week_cos', 'year_2018', 'year_2019',
       'year_2020', 'year_2021', 'year_2022', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'day_1', 'day_2', 'day_3', 'day_4',
       'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11',
       'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18',
       'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25',
       'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31',
       'Bumper Cars', 'Bungee Jump', 'Circus Train', 'Crazy Dance',
       'Dizzy Dropper', 'Drop Tower', 'Flying Coaster', 'Free Fall',
       'Giant Wheel', 'Giga Coaster', 'Go-Karts', 'Haunted House',
       'Himalaya Ride', 'Inverted Coaster', 'Kiddie Coaster

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold

# Define the number of splits for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store the results
train_rmse_list = []
val_rmse_list = []

cutoff_date = waiting_times['date'].max() - pd.DateOffset(months=3)
train_set = waiting_times[waiting_times['date'] < cutoff_date]
val_set = waiting_times[waiting_times['date'] >= cutoff_date]
X_train = train_set.drop(columns=['WAIT_TIME_MAX', 'date'])
y_train = train_set['WAIT_TIME_MAX']
X_val = val_set.drop(columns=['WAIT_TIME_MAX', 'date'])
y_val = val_set['WAIT_TIME_MAX']

# Perform cross-validation
for train_index, val_index in kf.split(train_set):
    X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize CatBoostRegressor
    model_cv = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, eval_metric='RMSE', random_seed=42)

    # Train the model
    model_cv.fit(X_train_cv, y_train_cv, eval_set=(X_val_cv, y_val_cv), verbose=100, early_stopping_rounds=50)

    # Predict on the validation set
    y_train_pred_cv = model_cv.predict(X_train_cv)
    y_val_pred_cv = model_cv.predict(X_val_cv)

    # Evaluate the model
    train_rmse = root_mean_squared_error(y_train_cv, y_train_pred_cv)
    val_rmse = root_mean_squared_error(y_val_cv, y_val_pred_cv)

    train_rmse_list.append(train_rmse)
    val_rmse_list.append(val_rmse)

# Print the average RMSE for training and validation sets
print(f'Average Training RMSE: {np.mean(train_rmse_list)}')
print(f'Average Validation RMSE: {np.mean(val_rmse_list)}')

# Train the final model on the entire training data
final_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, eval_metric='RMSE', random_seed=42)
final_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100, early_stopping_rounds=50)

# Predict on the final validation set
y_pred_final = final_model.predict(X_val)

# Evaluate the final model
final_rmse = root_mean_squared_error(y_val, y_pred_final)
print(f'Final Validation RMSE: {final_rmse}')

0:	learn: 18.8359477	test: 18.8178475	best: 18.8178475 (0)	total: 338ms	remaining: 5m 37s
100:	learn: 12.1463186	test: 12.1172524	best: 12.1172524 (100)	total: 20.9s	remaining: 3m 6s
200:	learn: 11.1672704	test: 11.1448370	best: 11.1448370 (200)	total: 40.8s	remaining: 2m 42s
300:	learn: 10.6668252	test: 10.6496117	best: 10.6496117 (300)	total: 1m 1s	remaining: 2m 21s
400:	learn: 10.3215150	test: 10.3087133	best: 10.3087133 (400)	total: 1m 21s	remaining: 2m 1s
500:	learn: 10.0357455	test: 10.0280297	best: 10.0280297 (500)	total: 1m 42s	remaining: 1m 41s


KeyboardInterrupt: 